<a href="https://colab.research.google.com/github/project0fightttt-bit/ensenble/blob/main/%EA%B5%B0%EC%A7%91%20%EB%B6%84%EC%84%9D_%ED%98%84%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 영화/TV 데이터 셋에서 NULL값은 제외한 데이터프레임

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
sets = {
    "A_duration_included": [
        "duration_weeks", "peak_strength", "top3_ratio", "rank_std"
    ],

    "B_shape_only": [
        "peak_strength", "top3_ratio", "rank_std", "ttp_ratio"
    ],

    "C_views_only": [
        "max_views", "mean_views", "decay_rate", "cv_views", "hype_ratio"
    ],

    "D_mixed_shape_views": [
        "peak_strength", "top3_ratio", "rank_std",
        "decay_rate", "cv_views"
    ]
}

In [6]:
# 1. 데이터 로드
url = 'https://github.com/project0fightttt-bit/ensenble/raw/refs/heads/main/all-weeks-global_no_nan_weekly_views.xlsx'
df = pd.read_excel(url)
df['week'] = pd.to_datetime(df['week'])

# 2. 작품별 변수(Feature) 생성
features_list = []
for name, group in df.groupby('show_title'):
    group = group.sort_values('week')

    # 기초 값들
    weeks = group['cumulative_weeks_in_top_10'].max()
    ranks = group['weekly_rank']
    views = group['weekly_views']

    # 변수 계산
    peak_strength = 11 - ranks.min()
    top3_count = len(group[group['weekly_rank'] <= 3])
    top3_ratio = top3_count / weeks
    rank_std = ranks.std() if len(ranks) > 1 else 0

    peak_week_idx = views.idxmax()
    weeks_to_peak = ((group.loc[peak_week_idx, 'week'] - group.iloc[0]['week']).days / 7) + 1
    ttp_ratio = weeks_to_peak / weeks

    max_views = views.max()
    mean_views = views.mean()
    cv_views = (views.std() / mean_views) if mean_views > 0 and len(views) > 1 else 0

    # 넷플릭스 보고서 취지 반영 변수
    efficiency_index = mean_views / group['runtime'].mean() if group['runtime'].mean() > 0 else 0
    viral_growth = max_views / views.iloc[0] if views.iloc[0] > 0 else 1

    features_list.append({
        'show_title': name,
        'duration_weeks': weeks,
        'peak_strength': peak_strength,
        'top3_ratio': top3_ratio,
        'rank_std': rank_std,
        'ttp_ratio': ttp_ratio,
        'max_views': max_views,
        'mean_views': mean_views,
        'cv_views': cv_views,
        'efficiency_index': efficiency_index,
        'viral_growth': viral_growth
    })

df_features = pd.DataFrame(features_list).fillna(0)

# 3. 변수 조합(Sets) 정의
sets = {
    "A_duration_included": ["duration_weeks", "peak_strength", "top3_ratio", "rank_std"],
    "B_shape_only": ["peak_strength", "top3_ratio", "rank_std", "ttp_ratio"],
    "C_views_only": ["max_views", "mean_views", "cv_views", "efficiency_index"],
    "D_mixed_shape_views": ["peak_strength", "top3_ratio", "rank_std", "viral_growth", "cv_views"]
}

# 4. 각 세트별로 k=2~8 실루엣 점수 계산
scaler = StandardScaler()

for set_name, cols in sets.items():
    print(f"\n==== {set_name} ====")
    X = df_features[cols]
    X_scaled = scaler.fit_transform(X)

    for k in range(2, 9):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        print(f"k={k}: {score:.4f}")


==== A_duration_included ====
k=2: 0.5237
k=3: 0.5297
k=4: 0.4284
k=5: 0.4285
k=6: 0.3962
k=7: 0.4085
k=8: 0.4004

==== B_shape_only ====
k=2: 0.5407
k=3: 0.5646
k=4: 0.4605
k=5: 0.4603
k=6: 0.4619
k=7: 0.4601
k=8: 0.4342

==== C_views_only ====
k=2: 0.6030
k=3: 0.4251
k=4: 0.3856
k=5: 0.4109
k=6: 0.4129
k=7: 0.4160
k=8: 0.4103

==== D_mixed_shape_views ====
k=2: 0.5265
k=3: 0.5332
k=4: 0.4256
k=5: 0.4487
k=6: 0.4019
k=7: 0.3714
k=8: 0.3774


In [7]:
# 💻 Set E, F, G 실루엣 점수 측정 코드


# 1. 데이터 로드
url = 'https://github.com/project0fightttt-bit/ensenble/raw/refs/heads/main/all-weeks-global_no_nan_weekly_views.xlsx'
df = pd.read_excel(url)
df['week'] = pd.to_datetime(df['week'])

# 2. 작품별 Feature Engineering (New Sets)
features_list = []
for name, group in df.groupby('show_title'):
    group = group.sort_values('week')

    # 기초 값 준비
    weeks = group['cumulative_weeks_in_top_10'].max()
    views = group['weekly_views']
    ranks = group['weekly_rank']
    runtime = group['runtime'].mean()
    first_views = views.iloc[0] if views.iloc[0] > 0 else 0.001
    max_views = views.max()
    mean_views = views.mean()

    # 피크 시점 계산
    peak_week_idx = views.idxmax()
    t_peak = ((group.loc[peak_week_idx, 'week'] - group.iloc[0]['week']).days / 7) + 1

    # 피크 이후 데이터
    after_peak_views = views[views.index >= peak_week_idx]
    mean_after_peak = after_peak_views.mean()

    # [변수 조합 생성]
    # Set E용
    hype_resilience = mean_views / first_views
    concentration = max_views / views.sum()
    rank_stability = 1 / (ranks.std() + 0.1) # 0 나누기 방지

    # Set F용
    viral_accel = (max_views - first_views) / t_peak if t_peak > 0 else 0
    peak_eff = max_views / runtime if runtime > 0 else 0
    ttp_rel = t_peak / weeks

    # Set G용
    eng_density = mean_views / runtime if runtime > 0 else 0
    survival_score = weeks * (11 - ranks.mean())
    late_tail = mean_after_peak / max_views

    features_list.append({
        'show_title': name,
        'hype_resilience': hype_resilience,
        'concentration': concentration,
        'rank_stability': rank_stability,
        'viral_accel': viral_accel,
        'peak_eff': peak_eff,
        'ttp_rel': ttp_rel,
        'eng_density': eng_density,
        'survival_score': survival_score,
        'late_tail': late_tail
    })

df_features = pd.DataFrame(features_list).fillna(0)

# 3. 새로운 변수 세트 정의
new_sets = {
    "E_Hype_vs_Heart": ["hype_resilience", "concentration", "rank_stability"],
    "F_Viral_Velocity": ["viral_accel", "peak_eff", "ttp_rel"],
    "G_Executive_View": ["eng_density", "survival_score", "late_tail"]
}

# Hype Resilience: 평균 시청 / 첫 주 시청 =	"오픈 빨이 빠진 뒤에도 관객이 남았는가?" (작품의 내공)
# Viral Accel: (피크 - 첫 주) / 기간 = "입소문이 얼마나 공격적으로 퍼졌는가?" (바이럴 파괴력)
# Concentration: 피크 시청 / 전체 시청 = "반짝 화제성인가, 꾸준한 사랑인가?" (팬덤의 두께)

# 4. 실루엣 점수 측정
scaler = StandardScaler()

for set_name, cols in new_sets.items():
    print(f"\n==== {set_name} ====")
    X = df_features[cols]
    X_scaled = scaler.fit_transform(X)

    for k in range(2, 6): # 실무적으로 가장 유의미한 2~5까지 확인
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        print(f"k={k}: {score:.4f}")


==== E_Hype_vs_Heart ====
k=2: 0.7220
k=3: 0.6263
k=4: 0.6345
k=5: 0.6257

==== F_Viral_Velocity ====
k=2: 0.7186
k=3: 0.7460
k=4: 0.7299
k=5: 0.5801

==== G_Executive_View ====
k=2: 0.4967
k=3: 0.5188
k=4: 0.5004
k=5: 0.3977
